# インシデント調査（LC4RI風）


TBD：前口上を書く


# 事前準備

環境変数で、時刻をJSTに、ラッパーカーネルの機能を有効にする。

In [ ]:
%env TZ JST-9
%env lc_wrapper_force on

必要なモジュールをインストールしておく。

In [ ]:
!sudo apt-get update
!sudo apt-get upgrade -y
!sudo apt-get install -y sysv-rc-conf iproute2 kmod lsof nmap netcat

In [ ]:
!pip install -U pip
!pip install pip-review
!sudo -H pip-review --auto
!pip install parse

システムの負荷状態を記録しておく。

In [ ]:
import datetime
JST = datetime.timezone(datetime.timedelta(hours=+9), 'JST')

import parse

uptime_list = []

uptime_str = !uptime
uptime_clumn = parse.parse("{}load average: {:g}, {:g}, {:g}", uptime_str[0])
uptime_clumn = list(uptime_clumn)[1:]
uptime_clumn.insert(0, datetime.datetime.now(JST))
uptime_list.append(uptime_clumn)
print(uptime_list)

# ライブレスポンス

参考：[インシデントレスポンス 第3版（日経BP社）](https://www.amazon.co.jp/dp/4822279871/) p179

但し、本来は、調査対象ホスト上で実行すべき作業であるが、本ノートブックでは簡易なデモとして、Jupyterのコンテナ自身に対して作業を行う。

<font color="Red">前提条件を満たしていないと失敗する確認用のセル（Design By Contract）</font>

使用するコマンド類が使えることを確認する。

In [ ]:
!which sysv-rc-conf
!which ip
!which lsmod
!which lsof

システムの日付と時間

In [ ]:
!date

<font color="Red">再利用性が高く、実行結果が安定している場合、実行結果をお手本に残す</font>

インストールされているソフトウェア

In [ ]:
%env lc_wrapper_force off

In [ ]:
!dpkg --get-selections

In [ ]:
%env lc_wrapper_force on

ファイルシステム情報

In [ ]:
!mount -l

In [ ]:
!df -Th

In [ ]:
!fdisk -l

OSのバージョン

In [ ]:
!cat /etc/issue

カーネルのバージョン

In [ ]:
!uname -a

稼働時間

In [ ]:
!w

cron

In [ ]:
!find /var/spool/cron -type f -print0 | xargs -0 more | cat

In [ ]:
!find /etc/cron.* -type f -print0 | xargs -0 more | cat

サービス

In [ ]:
!sudo sysv-rc-conf --list

ユーザアカウント

In [ ]:
!cat /etc/passwd

In [ ]:
!sudo cat /etc/shadow

グループ

In [ ]:
!cat /etc/group

ネットワークインタフェース

In [ ]:
!ip a

In [ ]:
!ip -s l

ルーティングテーブル

In [ ]:
!ip r

ARPテーブル

In [ ]:
!ip n

ネットワーク接続

TCPとUDPそれぞれ

In [ ]:
!ss -nat

In [ ]:
!ss -nau

ロードされているドライバ

In [ ]:
!lsmod

オープン中のファイルとハンドル

In [ ]:
!lsof

実行中のプロセスとスレッド

In [ ]:
!ps auxwwwem

設定情報の保全

In [ ]:
!rm -f host_etc.tar.gz ; sudo tar cvzf host_etc.tar.gz /etc/ ; ls -la host_etc.tar.gz

システムログの保全

In [ ]:
!rm -f host_log.tar.gz ; sudo tar cvzf host_log.tar.gz /var/log/ ; ls -la host_log.tar.gz

ファイル一覧

In [ ]:
!rm -f host_files.txt ; sudo find / -xdev -printf "%m;%Ax;%AT;%Tx;%TT;%Cx;%CT;%U;%G;%s;%p;\n" > host_files.txt
!ls -la host_files.txt

シェルの履歴をhistoryで取る必要があるが、本デモでは、Jupyterの仕様上、割愛する。

システムの負荷状態を記録しておく。

In [ ]:
uptime_str = !uptime
uptime_clumn = parse.parse("{}load average: {:g}, {:g}, {:g}", uptime_str[0])
uptime_clumn = list(uptime_clumn)[1:]
uptime_clumn.insert(0, datetime.datetime.now(JST))
uptime_list.append(uptime_clumn)
print(uptime_list)

# サーバの調査

## システムログを洗う

各種ログに含まれる「error」を確認する。

In [ ]:
!find /var/log/ -type f -print0 | sudo xargs -0 grep error

各種ログを他のキーワードでも検索してみよう。

In [ ]:
!find /var/log/ -type f -print0 | sudo xargs -0 grep 他のキーワード

システムの負荷状態を記録しておく。

In [ ]:
uptime_str = !uptime
uptime_clumn = parse.parse("{}load average: {:g}, {:g}, {:g}", uptime_str[0])
uptime_clumn = list(uptime_clumn)[1:]
uptime_clumn.insert(0, datetime.datetime.now(JST))
uptime_list.append(uptime_clumn)
print(uptime_list)

## httpサーバの動作を確認する

外との通信に使われているネットワークインタフェースについて、空いているポートの確認を行う。

<font color="Red">前提条件を満たしていないと失敗する確認用のセル（Design By Contract）</font>

使用するコマンド類が使えることを確認する。

In [ ]:
!which nmap
!which nc

In [ ]:
!nmap -p1-65535 eth0に付いているIPアドレス

空いてるポートがhttpに反応するか確認する。

In [ ]:
!echo "GET / HTTP/1.1\n\n" | nc -w 1 eth0に付いているIPアドレス 空いているポート番号

[HTTPステータスコード](https://ja.wikipedia.org/wiki/HTTP%E3%82%B9%E3%83%86%E3%83%BC%E3%82%BF%E3%82%B9%E3%82%B3%E3%83%BC%E3%83%89)を参考にhtmlのコンテンツが得られるまで追いかけてみる。

In [ ]:
!echo "GET 取得したいURL HTTP/1.1\n\n" | nc -w 1 eth0に付いているIPアドレス 空いているポート番号

In [ ]:
!echo "GET 取得したいURL HTTP/1.1\n\n" | nc -w 1 eth0に付いているIPアドレス 空いているポート番号

追いかけた後で、wgetの出力を見ると、ブラウザがコンテンツを取得するまでの振る舞いが理解できる。

In [ ]:
!wget -O - http://eth0に付いているIPアドレス:8888

システムの負荷状態を記録しておく。

In [ ]:
uptime_str = !uptime
uptime_clumn = parse.parse("{}load average: {:g}, {:g}, {:g}", uptime_str[0])
uptime_clumn = list(uptime_clumn)[1:]
uptime_clumn.insert(0, datetime.datetime.now(JST))
uptime_list.append(uptime_clumn)
print(uptime_list)

## 調査作業時のシステムの負荷状態を確認する

各作業終了時に[uptime](https://ja.wikipedia.org/wiki/Uptime)の結果をuptime_listに保存してきた。

In [ ]:
print(uptime_list)

これをグラフ化する。

In [ ]:
import pandas
import matplotlib

In [ ]:
%matplotlib inline

In [ ]:
df = pandas.DataFrame(uptime_list, columns=["date", "1m", "5m", "15m"])
df = df.set_index("date")
df

In [ ]:
df.plot()